In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            return 5000 + 5000 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            reward += -2
            
            if len(self.env.distances) > 0 and len(self.env.history) > 1:
                target_diff = self.env.distances[-1] - self.env.get_target_distance(False)
                
                gps_diff_0 = self.env.history[-2].gps_actual[0] - self.env.state.gps_actual[0]
                gps_diff_1 = self.env.history[-2].gps_actual[1] - self.env.state.gps_actual[1]
                
                #print("history: ", self.env.history[-1].gps_actual[0])
                #print("actual:  " , self.env.state.gps_actual[0])
                
                actual_diff = math.sqrt(gps_diff_0**2 + gps_diff_1**2)
                
                if actual_diff > 0:
                    diff_rew = 4 * ( target_diff  / actual_diff)
                else:
                    diff_rew = 0
                
                
                
                if abs(diff_rew) > 4:
                    print("============= Quotient > 1 !? ==========")
                    print("Target_diff: ", target_diff)
                    print("Actual_diff: ", actual_diff)
                    print("diff_rew:    ", diff_rew)
                    
                if diff_rew < 0:
                    diff_rew = diff_rew * 2
                
                #print("Target_diff: ", target_diff)
                #print("Actual_diff: ", actual_diff)
                #print("diff_rew:    ", diff_rew)  
                    
  
                reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -5
        
            if self.env.state.touching:
                reward += -100
        return reward

    def check_done(self):
        if self.env.total_reward < -5000:
            print("reward boundary, reward: ", self.env.total_reward)
            return True
        if self.env.get_target_distance(False) < 0.1:
            print("target reached, reward: ", self.env.total_reward)
            return True
        if self.env.total_reward > 25000:
            print("Stop hacking!, reward: ", self.env.total_reward)
            return True
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 10
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
#                 action_class = wg.WbtActDiscrete(config, dspeed=0.05, ddir=0.05),
                config=config)

../../backend/webotsgym/env/webotenv.py:111: UserWarning: Relative property of action class is overwritten by config.relative_action.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
model_name = "3x3_reach_target_v6"
model = PPO1("MlpPolicy", env, timesteps_per_actorbatch = 5000, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
while True:
    model.learn(total_timesteps=500000)
    model.save("./{}".format(model_name))


reward boundary, reward:  -5075.138390909345
reward boundary, reward:  -5013.254815767437
reward boundary, reward:  -5033.5091386599715
reward boundary, reward:  -5087.786258716982
reward boundary, reward:  -5024.045613093157
reward boundary, reward:  -5008.509655753781
reward boundary, reward:  -5054.38617389161
reward boundary, reward:  -5080.072414041448
reward boundary, reward:  -5100.005598459242
reward boundary, reward:  -5067.950279860842
reward boundary, reward:  -5011.904305615229
target reached, reward:  -1769.3919054887645
target reached, reward:  -1506.5026336757157

reward boundary, reward:  -5001.076360459701
reward boundary, reward:  -5003.74068047095
target reached, reward:  -2775.7471875119295
reward boundary, reward:  -5075.205207944889
reward boundary, reward:  -5104.994773197897
target reached, reward:  -4490.768425756203
reward boundary, reward:  -5004.264622656174
reward boundary, reward:  -5019.834311626317
target reached, reward:  -1318.1083992078989
reward bou

target reached, reward:  -1801.6668112116356
reward boundary, reward:  -5005.370643129012
target reached, reward:  -26.13245404180086
target reached, reward:  10.723849348179156
reward boundary, reward:  -5005.429028366687
reward boundary, reward:  -5037.600346670333
reward boundary, reward:  -5096.891715493432
reward boundary, reward:  -5031.034926587288
target reached, reward:  -1406.3511001691481
reward boundary, reward:  -5059.31741492684
target reached, reward:  -1807.6137520876023
reward boundary, reward:  -5020.189858048437
target reached, reward:  -2155.6087686127294
target reached, reward:  -2306.0624475616446
target reached, reward:  -579.3767082489529
reward boundary, reward:  -5065.743816013294
reward boundary, reward:  -5114.091098103273
target reached, reward:  -3516.1488047165
reward boundary, reward:  -5078.493407652511
target reached, reward:  -1663.1247570877745
reward boundary, reward:  -5038.911222730515
reward boundary, reward:  -5097.523006580834
reward boundary, 